# OkadaTest

Read in a fault model from the coarsened triangulation provided by Jey and apply the Okada model from GeoClaw to obtain the static seafloor deformation at the final time.

(Could be adapted to compute time-dependendent kinetic rupture based on applying Okada at each specified time to the subfaults that have ruptured up to that time, but that would require `rupture time` and `rise_time` for each subfault, which are not included currently in Jey's coarsened model.)


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import dtopotools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from copy import copy
import os,sys

## Read in the fault geometry:

And specify a `dtopotools.Fault` object with this geometry.

In [ ]:
datadir = '/Users/rjl/D/JeysCode/Audreys_source_models_interpolation_09182024/'
triangles = loadtxt(datadir+'David_cas_fine_mesh.tri')  # list of triangles, 3 vertices
vertices = loadtxt(datadir+'David_cas_fine_mesh.ned')  # list of vertices, lon-lat of each

vertices[:,1] = vertices[:,1] - 360.  # shift to longitude W
vertices[:,3] = -1000*vertices[:,3]   # convert depth to positive depth in meters

In [ ]:
fault0 = dtopotools.Fault(coordinate_specification='triangular')
fault0.subfaults = []

nsubfaults = triangles.shape[0]

for j in range(nsubfaults):
    subfault0 = dtopotools.SubFault()
    jv = int(triangles[j,1]) - 1
    node1 = vertices[jv,1:4]
    jv = int(triangles[j,2]) - 1
    node2 = vertices[jv,1:4]
    jv = int(triangles[j,3]) - 1
    node3 = vertices[jv,1:4]  
    node_list = [node1, node2, node3]
    subfault0.set_corners(node_list,projection_zone='10')
    fault0.subfaults.append(subfault0) 

print('Set up fault0 model with %i subfaults, without yet specifying slip for particular event' \
            % nsubfaults)

## Check that orientation of triangles are all correct:

In [ ]:
numpos = 0.
for s in fault0.subfaults:
    c = array(s.corners)[:3,:2]
    A = vstack([c[:,0], c[:,1], array([1,1,1])]).T
    detA = np.linalg.det(A)
    if detA > 0:
        numpos += 1
if numpos > 0:
    print('*** Warning, %i of the %i subfaults have counterclockwise orientation' \
            % (numpos, nsubfaults))
else:
    print('All subfault triangles have clockwise orientation')

## Plot triangulation:

In [ ]:
fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(121, projection='3d')
ax = fig.add_axes([.05,.05,.9,.9], projection='3d')
for s in fault0.subfaults:
    c = s.corners
    c.append(c[0])
    c = np.array(c)
    ax.plot(c[:,0],c[:,1],-c[:,2]/1000.,color='b')
ax.view_init(10,60)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Depth (km)')
ax.set_title('Triangular subfaults')

#ax = fig.add_subplot(122)
#ax = fig.add_axes([.75,.05,.2,.9])
fig = figure(figsize=(10,15))
ax = axes()
for s in fault0.subfaults:
    c = s.corners
    c.append(c[0])
    c = np.array(c)
    ax.plot(c[:,0],c[:,1], 'b')
ax.set_aspect(1./np.cos(45*np.pi/180.))
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Plan view');

## Functions for setting up an event and applying Okada:

In [ ]:
def set_slip(fault0, event):
    import copy
    fault = copy.copy(fault0)
    
    event_dir = datadir + 'output_files_orig'
    
    if 0:
        # don't need slip in dip and strike directions separately, only magnitude read below
        fname = 'dip_slip_resampled_source_saved_%s.out' % event
        dip_slip = loadtxt(os.path.join(event_dir, fname))
        fname = 'strike_slip_resampled_source_saved_%s.out' % event
        strike_slip = loadtxt(os.path.join(event_dir, fname))
        
    fname = 'mag_slip_resampled_source_saved_%s.out' % event
    mag_slip = loadtxt(os.path.join(event_dir, fname))[:,2]
    
    fname = 'rake_resampled_source_saved_%s.out' % event
    rake = loadtxt(os.path.join(event_dir, fname))[:,2]
    
    for j in range(nsubfaults):
        subfault = fault.subfaults[j]
        subfault.rake = rake[j]
        subfault.slip = mag_slip[j]
        
    print('Created fault with Mw = %.2f' % fault.Mw())
    
    fault.event = event  # save name of event in fault object
    return fault


def make_dtopo(fault, times=[0.]):
    if len(times) <= 2:
        fault.rupture_type = 'static'
    else:
        fault.rupture_type = 'kinematic'

    dx = dy = 30/3600.  # spatial resolution for dtopo file
    #x,y = fault.create_dtopo_xy(dx=dx)  # choose automatically
    # use same x,y as in dtopo files made from ground motions:
    x = arange(-128.5, -122.4999, dx)
    y = arange(40,50.0001,dy)
    
    print('Will create dtopo on arrays of shape %i by %i ...' % (len(x),len(y)))
    dtopo = fault.create_dtopography(x,y,times=times,verbose=100);
    
    fname = 'dtopo_%s.dtt3' % fault.event
    dtopo.write(fname, dtopo_type=3)
    print('Created %s with %s displacement at %i times' \
            % (fname, fault.rupture_type, len(times)))
    return dtopo

def plot_slip_final_dtopo(fault, dtopo):

    """
    Plot dtopo (at final time for kinematic rupture).
    """
    
    fig,(ax0,ax1) = plt.subplots(ncols=2,nrows=1,figsize=(12,6))
    fault.plot_subfaults(axes=ax0,slip_color=True,plot_box=False);
    ax0.set_title('Slip on Fault');

    X = dtopo.X; Y = dtopo.Y; dZ_at_t = dtopo.dZ_at_t
    dz_max = dtopo.dZ.max()
    tfinal = dtopo.times[-1] + 1  # 1 second after final dZ
    dtopotools.plot_dZ_colors(X,Y,dZ_at_t(tfinal),axes=ax1, 
                              cmax_dZ = dz_max, 
                              dZ_interval=200, add_colorbar=True);
    ax1.set_title('Seafloor deformation (static Okada)');
    

## Test on one event:

In [ ]:
event = 'locking_mur13_deep'
fault = set_slip(fault0, event)
print('There are %i subfaults in this model' % len(fault.subfaults))

### Check how many have small slip:

In [ ]:
slips = array([abs(s.slip) for s in fault.subfaults])
for sliptol in [1e-3, 1e-2, 1e-1]:
    ii = where(slips > sliptol)[0]
    print('%i subfaults have abs(slip) > %.3f m' % (len(ii),sliptol))

## Try first on small subset, runs more quickly...

In [ ]:
testfault = dtopotools.Fault(coordinate_specification='triangular')
testfault.subfaults = []
for s in fault.subfaults:
    if 45.4<s.latitude<45.8 and -126<s.longitude<-125:
        testfault.subfaults.append(s)
print('Created testfault with %i subfaults' % len(testfault.subfaults))
testfault.event = 'testfault'
testdtopo = make_dtopo(testfault, times=[0.])


In [ ]:
fig,(ax0,ax1) = plt.subplots(ncols=2,nrows=1,figsize=(12,6))
xlim = (-125.7, -124.9)
ylim = (45.3, 45.9)
testfault.plot_subfaults(axes=ax0,slip_color=True,plot_box=False);
ax0.set_title('Slip on Fault');
ax0.set_xlim(xlim)
ax0.set_ylim(ylim)

#X = dtopo.X; Y = dtopo.Y; dZ_at_t = dtopo.dZ_at_t
dz_max = testdtopo.dZ.max()
testdtopo.plot_dZ_colors(t=2, axes=ax1, 
                          cmax_dZ = dz_max, 
                          dZ_interval=2);

ax1.set_xlim(xlim)
ax1.set_ylim(ylim)
ax1.set_title('Seafloor deformation (static Okada)');

## Run Okada on full rupture:

In [ ]:
dtopo = make_dtopo(fault, times=[0.])

In [ ]:
plot_slip_final_dtopo(fault, dtopo)

## Compare to final displacement from seismic simulation:

In [ ]:
dtopodir = '../dtopofiles/'
dtopo_instant_fname = dtopodir + 'buried-%s_instant.dtt3' % event.replace('_','-')
print('Comparing to %s' % dtopo_instant_fname)

In [ ]:
dtopo_instant = dtopotools.DTopography(dtopo_instant_fname, dtopo_type=3)

In [ ]:
fig,(ax0,ax1) = plt.subplots(ncols=2,nrows=1,figsize=(12,6))

X = dtopo_instant.X; Y = dtopo_instant.Y; dZ_at_t = dtopo_instant.dZ_at_t
dz_max = dtopo_instant.dZ.max()
tfinal = dtopo_instant.times[-1] + 1  # 1 second after final dZ
dtopotools.plot_dZ_colors(X,Y,dZ_at_t(tfinal),axes=ax0, 
                          cmax_dZ = dz_max, 
                          dZ_interval=200, add_colorbar=True);
ax0.set_title('Seafloor deformation (static seismic)');
ax0.set_ylim(40,50)
ax0.set_xlim(-128.5,-122)

X = dtopo.X; Y = dtopo.Y; dZ_at_t = dtopo.dZ_at_t
#dz_max = dtopo.dZ.max() # use same color scale as in other figure
tfinal = dtopo.times[-1] + 1  # 1 second after final dZ
dtopotools.plot_dZ_colors(X,Y,dZ_at_t(tfinal),axes=ax1, 
                          cmax_dZ = dz_max, 
                          dZ_interval=200, add_colorbar=True);
ax1.set_title('Seafloor deformation (static Okada)');
ax1.set_ylim(40,50)
ax1.set_xlim(-128.5,-122)